<a href="https://colab.research.google.com/github/ambarja/INLA-MODELS/blob/main/inla_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" style="padding-right:10px;" src="https://avatars.githubusercontent.com/u/46831228?s=200&v=4" width=10% ><img align="right" style="padding-left:10px;" src="https://colab.research.google.com/img/colab_favicon.ico" width=10% >
<p><b><h2 align ="center">Malaria Hydrobasins project</h2></b></p>
<p>Este proyecto tiene como objetivo analizar de forma espacial y temporal las tasas de malaria en la región de Loreto a nivel distrital y microcuenas empleando modelos bayesianos como INLA. <img src = 'https://img.shields.io/github/license/Naereen/StrapDown.js.svg' href='https://github.com/Naereen/StrapDown.js/blob/master/LICENSE'> </p>


In [ ]:
#@title **Install packages main. It takes aprox. 10 min**
system('sudo add-apt-repository ppa:ubuntugis/ubuntugis-unstable')
system('sudo apt-get update')
system('sudo apt-get install libudunits2-dev libgdal-dev libgeos-dev libproj-dev')
system('sudo apt-get install libprotobuf-dev protobuf-compiler libv8-dev libjq-dev')
install.packages('sf')
install.packages('tidyverse')
install.packages("INLA",repos=c(getOption("repos"),INLA="https://inla.r-inla-download.org/R/testing"), dep=TRUE)
install.packages('spdep')
install.packages('Metrics')
install.packages('spatialreg')

In [1]:
#@title **functions - inla_utils**
inla_function <- function(data, eq) {
  result <- inla(eq,
    family = "nbinomial",
    verbose = F,
    data = data,
    E = nrohab,
    control.compute = list(dic = TRUE, waic = TRUE, cpo = TRUE),
    control.predictor = list(compute = TRUE, link = 1)
  )
}


metricas_error <- function(data, model){
  data$pred <- model$summary.fitted.values$`0.5quant`
  met_rmse <- rmse(actual = data$fal , predicted = data$pred)
  met_rse  <- rse(actual = data$fal , predicted = data$pred)
  met_msle <- msle(actual = data$fal , predicted = data$pred)
  met_mae <- mae(actual = data$fal , predicted = data$pred)
  met_tabla <- tibble(rmse = met_rmse, rse = met_rse, msle = met_msle, met_mae)
  return(met_tabla)
}


In [3]:
library(sf)
library(INLA)
library(tidyverse)
library(purrr)
library(spdep)
library(Metrics)

##1.Reading to dataset and spatial data

In [4]:
spatial <- read_rds("/content/sp_district.rds")
dataset <- read_rds("/content/alldataset_district.rds") %>% mutate(month = as.integer(month))

##2.Calculate the neighborhood matrix

In [5]:
nb <- poly2nb(spatial)
g <- inla.read.graph(nb)

Registered S3 methods overwritten by 'spatialreg':
  method                   from 
  residuals.stsls          spdep
  deviance.stsls           spdep
  coef.stsls               spdep
  print.stsls              spdep
  summary.stsls            spdep
  print.summary.stsls      spdep
  residuals.gmsar          spdep
  deviance.gmsar           spdep
  coef.gmsar               spdep
  fitted.gmsar             spdep
  print.gmsar              spdep
  summary.gmsar            spdep
  print.summary.gmsar      spdep
  print.lagmess            spdep
  summary.lagmess          spdep
  print.summary.lagmess    spdep
  residuals.lagmess        spdep
  deviance.lagmess         spdep
  coef.lagmess             spdep
  fitted.lagmess           spdep
  logLik.lagmess           spdep
  fitted.SFResult          spdep
  print.SFResult           spdep
  fitted.ME_res            spdep
  print.ME_res             spdep
  print.lagImpact          spdep
  plot.lagImpact           spdep
  summary.lagImpact      

##3.Models spatial, temporal and spatial-*temporal*

In [6]:
types <- c(
  # model random iid
  formula = fal ~ esco + etp + evi + hsoil + ndvi + pp + savi + temp +
    f(new_id, model = "iid", graph = g),

  # model spatial type besagproper
  formula = fal ~ esco + etp + evi  + hsoil + ndvi + pp + savi + temp +
    f(new_id, model = "besagproper", graph = g),

  # model spatial type besag
  formula = fal ~ esco + etp + evi + hsoil + ndvi + pp + savi + temp +
    f(new_id, model = "besag", graph = g),

  # model spatial type bym
  formula = fal ~ esco + etp + evi + hsoil + ndvi + pp + savi + temp +
    f(new_id, model = "bym", graph = g),

  # model temporal ar1
  formula = fal ~ esco + etp + evi  + hsoil + ndvi + pp + savi + temp +
    f(new_id, model = "ar1", graph = g),

  # model temporal rw1
  formula = fal ~ esco + etp + evi  + hsoil + ndvi + pp + savi + temp +
    f(new_id, model = "rw1", graph = g),

  # model temporal rw2
  formula = fal ~ esco + etp + evi  + hsoil + ndvi + pp + savi + temp +
    f(new_id, model = "rw2", graph = g),

  # model spatial-temporal (besagproper-rw1)
  formula = fal ~ esco + etp + evi  + hsoil + ndvi + pp + savi + temp +
    f(new_id, model = "besagproper", graph = g) + f(year, model = "rw1") + f(month, model = "iid"),

  # model spatial-temporal (besag - rw1)
  formula = fal ~ esco + etp + evi  + hsoil + ndvi + pp + savi + temp +
    f(new_id, model = "besag", graph = g) + f(year, model = "rw1") + f(month, model = "iid"),

  # model spatial-temporal (bym - rw1)
  formula = fal ~ esco + etp + evi + hsoil + ndvi + pp + savi + temp +
    f(new_id, model = "bym", graph = g) + f(year, model = "rw1") + f(month, model = "iid"),

  # model spatial-temporal (besagproper-rw2)
  formula = fal ~ esco + etp + evi + hsoil + ndvi + pp + savi + temp +
    f(new_id, model = "besagproper", graph = g) + f(year, model = "rw2") + f(month, model = "iid"),

  # model spatial-temporal (besag - rw2)
  formula = fal ~ esco + etp + evi + hsoil + ndvi + pp + savi + temp +
    f(new_id, model = "besag", graph = g) + f(year, model = "rw2") + f(month, model = "iid"),

  # model spatial-temporal (bym - rw2)
  formula = fal ~ esco + etp + evi  + hsoil + ndvi + pp + savi + temp +
    f(new_id, model = "bym", graph = g) + f(year, model = "rw2") + f(month, model = "iid")
)

##4.Names of models

In [7]:
model_name <- c("iid", "besagproper", "besag", "bym", "ar1", "rw1", "rw2",
  "besagproper-rw1", "besag-rw1", "bym-rw1", "besagproper-rw2",
  "besag-rw2", "bym-rw2")
names(types) <- model_name

##5.Run models

In [8]:
model_types <- types %>% purrr::map(~ inla_function(data = dataset, eq = .))

Warning message in inla.model.properties.generic(inla.trim.family(model), mm[names(mm) == :
“Model 'besagproper' in section 'latent' is marked as 'experimental'; changes may appear at any time.
  Use this model with extra care!!! Further warnings are disabled.”


In [9]:
class(model_types)

[1] "list"

##6.Metrics

In [10]:
model_metricas <- model_types %>% 
  purrr::map(~ metricas_error(data = dataset,model = .))

In [11]:
model_metricas

rmse,rse,msle,met_mae
<dbl>,<dbl>,<dbl>,<dbl>
37.01142,1.092597,3.025404,10.78031
rmse,rse,msle,met_mae
<dbl>,<dbl>,<dbl>,<dbl>
37.01142,1.092597,3.025416,10.78031
rmse,rse,msle,met_mae
<dbl>,<dbl>,<dbl>,<dbl>
37.01142,1.092597,3.025419,10.78031
rmse,rse,msle,met_mae
<dbl>,<dbl>,<dbl>,<dbl>


##7.Export models and metrics

In [12]:
saveRDS(model_types,'/content/resultados_distritos_inla.rds')
saveRDS(model_metricas,'/content/metricas_distritos_inla.rds')